- Reference
  - https://qiita.com/supersaiakujin/items/b9c9da9497c2163d5a74

In [1]:
import numpy as np
import os.path

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
#from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
#from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [3]:
batch_size = 128
nb_classes = 10
nb_epoch = 3

img_rows = 28
img_cols = 28

f_log = 'data/log'
f_model = 'data/model'
#model_filename = 'cnn_model.json'
#weights_filename = 'cnn_model_weights.hdf5'

In [11]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
#X_test  = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
# to NHWC
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test  = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [12]:
Y_train = to_categorical(y_train, nb_classes)
Y_test = to_categorical(y_test, nb_classes)

In [13]:
print(X_train.shape)
print(X_train.shape[1:])

(60000, 28, 28, 1)
(28, 28, 1)


In [15]:
model = Sequential()

#model.add(Convolution2D(32, 3, 3, border_mode = 'valid', input_shape=(1, img_rows, img_cols)))
#model.add(Conv2D(32, (3, 3), padding='valid', input_shape=X_train.shape[1:], data_format='channels_first'))
model.add(Conv2D(32, (3, 3), padding='valid', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Conv2D(64, (3, 3), padding='valid', data_format='channels_first'))
model.add(Conv2D(64, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.5), metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
activation_5 (Activation)    (None, 11, 11, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)             

In [16]:
history = model.fit(X_train,
                    Y_train,
                    batch_size=batch_size,
                    #nb_epoch=nb_epoch,
                    epochs=nb_epoch,
                    verbose=1,
                    #callbacks=cbks,
                    validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/3
469/469 [==============================] - 19s 40ms/step - loss: 0.1930 - accuracy: 0.9430 - val_loss: 0.0575 - val_accuracy: 0.9807
Epoch 2/3
469/469 [==============================] - 20s 43ms/step - loss: 0.0555 - accuracy: 0.9827 - val_loss: 0.0431 - val_accuracy: 0.9858
Epoch 3/3
469/469 [==============================] - 20s 43ms/step - loss: 0.0386 - accuracy: 0.9877 - val_loss: 0.0376 - val_accuracy: 0.9878
Test score: 0.0375753678381443
Test accuracy: 0.9878000020980835


In [18]:
print('save the architecture of a model')
json_string = model.to_json()
open(os.path.join(f_model, 'cnn_model.json'), 'w').write(json_string)

yaml_string = model.to_yaml()
open(os.path.join(f_model, 'cnn_model.yaml'), 'w').write(yaml_string)

print('save weights')
model.save_weights(os.path.join(f_model, 'cnn_model_weights.hdf5'))

save the architecture of a model
save weights
